In [1]:
import import_ipynb
import os
import math
import numpy as np
import datetime as dt
from numpy import newaxis
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import TensorBoard
from utils import Timer

Using TensorFlow backend.


importing Jupyter notebook from utils.ipynb


In [ ]:
class Model():
    """A class for an building and inferencing an lstm model"""
    def __init__(self):
        self.model = Sequential()
    def load_model(self, filepath):
        print('[Model] Loading model from file %s' % filepath)
        self.model = load_model(filepath)
    def build_model(self, configs):
        timer = Timer()
        timer.start()
        for layer in configs['model']['layers']:
            neurons = layer['neurons'] if 'neurons' in layer else None
            dropout_rate = layer['rate'] if 'rate' in layer else None
            activation = layer['activation'] if 'activation' in layer else None
            return_seq = layer['return_seq'] if 'return_seq' in layer else None
            input_timesteps = layer['input_timesteps'] if 'input_timesteps' in layer else None
            input_dim = layer['input_dim'] if 'input_dim' in layer else None
            if layer['type'] == 'dense':
                self.model.add(Dense(neurons, activation=activation))
            if layer['type'] == 'lstm':
                self.model.add(LSTM(neurons, input_shape=(input_timesteps, input_dim),return_sequences=return_seq))
            if layer['type'] == 'dropout':
                self.model.add(Dropout(dropout_rate))
        self.model.compile(loss=configs['model']['loss'], optimizer=configs['model']['optimizer'])
        print('[Model] Model Compiled')
        timer.stop()
    
    def model_to_json(self,save_dir):
        model_json = self.model.to_json()
        fname = os.path.join(save_dir, 'model.json')
        with open(fname, "w") as json_file:
            json_file.write(model_json)
        print('[Model] Serialize model to JSON at %s' % fname)
            
    def train(self, x, y, epochs, batch_size, save_dir):
        timer = Timer()
        timer.start()
        print('[Model] Training Started')
        print('[Model] %s epochs, %s batch size' % (epochs, batch_size))
        
        save_fname = os.path.join(save_dir, '%s-e%s.h5' % (dt.datetime.now().strftime('%d%m%Y-%H%M%S'), str(epochs)))
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=2),
            ModelCheckpoint(filepath=save_fname, monitor='loss', save_best_only=True)
        ]
        self.model.fit(
            x,
            y,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks
        )
        self.model.save(save_fname)
        print('[Model] Training Completed. Model saved as %s' % save_fname)
        timer.stop()
        
    def train_generator(self,data_gen,val_gen,epochs, batch_size,steps_per_epoch,log_fname,save_fname):
        timer = Timer()
        timer.start()
        print('[Model] Training Started')
        print('[Model] %s epochs, %s batch size, %s batches per epoch' % (epochs, batch_size,steps_per_epoch))

        callbacks = [
            ModelCheckpoint(filepath=save_fname, monitor='val_loss', save_best_only=True),
            TensorBoard(log_dir=log_fname, histogram_freq=0,write_graph=True, write_images=True)
        ]
        self.model.fit_generator(
            data_gen,
            validation_data=val_gen,
            validation_steps=1,
            steps_per_epoch=steps_per_epoch,
            epochs=epochs,
            callbacks=callbacks,
#             use_multiprocessing = True,
            workers=1
        )
        
        print('[Model] Training Completed. Model saved as %s' % save_fname)
        timer.stop()
        
    def predict_point_by_point(self, data):
        #Predict each timestep given the last sequence of true data, in effect only predicting 1 step ahead each time
        print('[Model] Predicting Point-by-Point...')
        predicted = self.model.predict(data)
#         print('before the predicted size is', predicted.shape )
        predicted = np.reshape(predicted, (predicted.size,))
        return predicted
    
    def evaluate(self, x_test, y_test):
        score = self.model.evaluate(x=x_test, y=y_test)
        return score
    
    def predict_sequences_multiple(self, data, window_size, prediction_len):
        #Predict sequence of 50 steps before shifting prediction run forward by 50 steps
        print('[Model] Predicting Sequences Multiple...')
        prediction_seqs = []
#         print('length of test data',len(data), 'prediction length:', prediction_len)
        for i in range(int(len(data)/prediction_len)):
            curr_frame = data[i*prediction_len]
            
            predicted = []
            for j in range(prediction_len):
#                 print(self.model.predict(curr_frame[newaxis,:,:]))
#                 print(self.model.predict(curr_frame[newaxis,:,:])[0,0])
                predicted.append(self.model.predict(curr_frame[newaxis,:,:])[0,0])
                
                curr_frame = curr_frame[1:]
                curr_frame = np.insert(curr_frame, [window_size-2], predicted[-1], axis=0)
            prediction_seqs.append(predicted)
        return prediction_seqs
    
    def predict_sequence_full(self, data, window_size):
        #Shift the window by 1 new prediction each time, re-run predictions on new window
        print('[Model] Predicting Sequences Full...')
        curr_frame = data[0]
        predicted = []
        for i in range(len(data)):
            predicted.append(self.model.predict(curr_frame[newaxis,:,:])[0,0])
            curr_frame = curr_frame[1:]
            curr_frame = np.insert(curr_frame, [window_size-2], predicted[-1], axis=0)
        return predicted